In [1]:
# https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/TensorFlow/Basics/tutorial4-convnet.py

import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
      for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            # other setting unrelated to this but who cares
            physical_devices = tf.config.list_physical_devices("GPU")
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
      print("No GPU device found")

No GPU device found


In [3]:
# 50000 training images
# 10000 test images
# 32x32 pixels rgb
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [4]:
model = keras.Sequential(
    [
        # 32 in height and width, 3 colors for rgb
        keras.Input(shape=(32, 32, 3)),
        # 32 channels, 3x3 kernel size for convolving crap.
        # padding=valid means that the output layer of this conv2d is going to be slightly smaller, in this case, it will become 30x30.
        layers.Conv2D(32, 3, padding="valid", activation="relu"),
        # pooling reduces the dimensions of a feature map. ^
        # default pool_size is (2, 2), which will halve the above layer (30x30 / 2 = 15x15)
        layers.MaxPooling2D(),
        # doubled channel size because... i don't know, it gets larger and more detailed at the bottom.
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        # flattening is done to convert the 2d vector into a single long continuous linear vector.
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        # i'm not really sure why it's 10
        layers.Dense(10),
    ]
)

In [5]:
# this is an application of the function api used in the previous notebook.
def my_model():
    inputs = keras.Input(shape=(32, 32, 3))
    # no activation function here, because that seems like a prerequisite whenever we're going to use batch normalization.
    # send something to a convolutional area first, then batch norm, then the activaiton function.
    x = layers.Conv2D(32, 3)(inputs)
    # this regularizes your data
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
model = my_model()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"]
)

In [7]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 25s - loss: 1.3286 - accuracy: 0.5273 - 25s/epoch - 32ms/step
Epoch 2/10
782/782 - 26s - loss: 0.9401 - accuracy: 0.6690 - 26s/epoch - 33ms/step
Epoch 3/10
782/782 - 27s - loss: 0.7914 - accuracy: 0.7229 - 27s/epoch - 34ms/step
Epoch 4/10
782/782 - 29s - loss: 0.6861 - accuracy: 0.7608 - 29s/epoch - 38ms/step
Epoch 5/10
782/782 - 26s - loss: 0.6005 - accuracy: 0.7920 - 26s/epoch - 34ms/step
Epoch 6/10
782/782 - 29s - loss: 0.5364 - accuracy: 0.8148 - 29s/epoch - 36ms/step
Epoch 7/10
782/782 - 30s - loss: 0.4725 - accuracy: 0.8373 - 30s/epoch - 39ms/step
Epoch 8/10
782/782 - 30s - loss: 0.4134 - accuracy: 0.8589 - 30s/epoch - 38ms/step
Epoch 9/10
782/782 - 31s - loss: 0.3648 - accuracy: 0.8764 - 31s/epoch - 39ms/step
Epoch 10/10
782/782 - 29s - loss: 0.3197 - accuracy: 0.8919 - 29s/epoch - 37ms/step
157/157 - 3s - loss: 0.9381 - accuracy: 0.7161 - 3s/epoch - 20ms/step


[0.9381260871887207, 0.7160999774932861]